# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
# replace the code below with your pulse rate algorithm.
import sys
!{sys.executable} -m pip install --upgrade numpy

#-------#
# Model #
#-------#

import glob

import numpy as np
import scipy as sp
import scipy.signal
import scipy.io

from matplotlib import pyplot as plt


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)
    
    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate(model):
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = model.RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)




#-------#
# Model #
#-------#
class PulseRateModel:
    """
        PulseRateModel is an object which estimates, predicts the pulse rate of 
        a given ppg signal and a three-axis accelerometer
    
    """
    def __init__(self, fs = 125, 
                       BPM_min = 40, 
                       BPM_max = 240,
                       pass_band = (2 / 3, 4),
                       window_length = 8,
                       window_shift = 2,
                       k_dominant_frequencies = 4):
        
        self.fs = fs  # signals were sampled at 125 Hz
        self.BPM_min = BPM_min # min bpm
        self.BPM_max= BPM_max # max bpm
        
        # self.pass_band = (self.BPM_min/self.fs, self.BPM_max/self.fs * 3)
        self.pass_band = pass_band
        self.window_length = window_length * self.fs # 8 second time window
        self.window_shift = window_shift * self.fs  # 2 second shift to next window
        
        self.k_dominant_frequencies = k_dominant_frequencies

    def read_mat_file(self, data_fl):
        """
            Small wrapper around the LoadTroikaDataFile methode,
            to make it more realistic
        """
        return LoadTroikaDataFile(data_fl)
    
    
    def bandpass_filter(self, signal, order=3):
        """filter the signal between 40 and 240 BPM

        Input:
            signal: input signal
            fs: Hz of input signal

        Output:
            filtered signal
        """

        b, a = scipy.signal.butter(order, self.pass_band, btype='bandpass', fs=self.fs)
        return scipy.signal.filtfilt(b, a, signal)
    
    
    
    def fourier_transformation(self, signal, n=2):
        """Calculate the fourier transform for each row in the signal matrix
           + return the fourier transformed frequencies

           input:
               signal a 2D signal matrix

            output:
                fft: N fourier transformations 
                freqs = frequency of the fourier transformation
        """

        # fourier transformation magnitude
        fft  = np.abs( np.fft.rfft(signal, n * signal.shape[1]) )

        # frequency
        freqs = np.fft.rfftfreq(n * signal.shape[1], 1 / self.fs)
        freqs = np.repeat(freqs[None,:], signal.shape[0], axis=0)
        return fft, freqs

    
    def calculate_confidence(self, freqs, fft_f, bpm_max):
        """calculates the confidence value for a signal window

        Args:
            freqs : list of frequenqies
            fft_f : fourier transformed signal
            bpm_max: max frequency

        Returns:
            confidence value (float64)
        """
        fundamental_freq_window = (freqs > (bpm_max - self.BPM_min/60)) & (freqs < bpm_max + self.BPM_min/60)
        fft_t_sum = fft_f.sum(axis=1)
        fft_f[fundamental_freq_window == False] = 0
        return fft_f.sum(axis=1) / fft_t_sum


    def RunPulseRateAlgorithm(self, data_fl, ref_fl):
        """
        RunPulseRateAlgorithm is the main method to calculate the pulse rate.
        It combines all the small steps together into one algorithm.
        
        - reads the data
        - applies a bandpass filter
        - combines the three-axis accelerometer into 1 signal
        - creates data windows of the filtered signals
        - calculates the fourier transformations
        - filters the fourier transformations
        - selects the most potential frequencies
        - calculates the confidence and error
        
        Args:
            data_fl : filename of the signal data
            ref_fl : filename of the reference data
        Returns:
            error (MAE)
            confidence
        """
       
        # Load data using LoadTroikaDataFile
        ppg, accx, accy, accz = self.read_mat_file(data_fl)

        # Compute pulse rate estimates and estimation confidence.
        # apply bandpass_filter to clean the signal
        ppg = self.bandpass_filter(ppg, order=2)
        #accx = self.bandpass_filter(accx, order=2)
        #accy = self.bandpass_filter(accy, order=2)
        #accz = self.bandpass_filter(accz, order=2)
        
        # Combine the accelerometer into 1 accelerometer signal - magniture
        acc = (accx**2 + accy**2 + accz**2)**0.5
        acc = self.bandpass_filter(acc, order=2)
        
        # create ppg and acc windows
        # loop over the signal in steps of 2 x fs (window_shift) --> (0 250 500 750 1000 ...)
        # for each step --> take the first n x fs values (250 251 252 ...  250+N)
        # (there is enough data to use full windows)
        assert(acc.shape == ppg.shape)
        # Create Subsets of the whole series
        ppg_windows = np.array([(ppg[ i : i + self.window_length], 
                                 acc[ i : i + self.window_length])  for i in range(0, ppg.shape[0] - self.window_length, self.window_shift)\
                                                                     if i + self.window_length < ppg.shape[0]])

        # split the window into acc_windows & ppg_windows
        acc_windows = ppg_windows[:,1,:]
        ppg_windows = ppg_windows[:,0,:]
        
        
        # fft the ppg and acc signals        
        fft_ppg, ppg_freqs = self.fourier_transformation(ppg_windows, n=2)
        fft_acc, acc_freqs = self.fourier_transformation(acc_windows, n=2)

                
        # filter the signals
        fft_ppg[ppg_freqs <= (self.BPM_min / 60)] = 0
        fft_ppg[ppg_freqs >= (self.BPM_max / 60)] = 0

        fft_acc[acc_freqs <= (self.BPM_min / 60)] = 0
        fft_acc[acc_freqs >= (self.BPM_max / 60)] = 0
        
        #
        # Calculate the take the dominant ppg frequency
        # 
        
        k = self.k_dominant_frequencies
        
        # take k first most dominant frequencies
        # sort the frequencies matrices by the row axis (1)
        # take the last 4 frequencies
        # revert the order (the most frequent frequency is now at index 0)
        k_dominant_ppg_freqs =  np.take_along_axis(ppg_freqs, np.argsort(fft_ppg, axis=1)[:,-k:], axis=1)[:,::-1]
        k_dominant_acc_freqs =  np.take_along_axis(acc_freqs, np.argsort(fft_acc, axis=1)[:,-k:], axis=1)[:,::-1]

        # Select the the dominant ppg freq, which isn't the same as the most dom acc freq
        # create a matrix with size (N_windows x k)
        # [[0, 1, 2, 3],
        #  [0, 1, 2, 3],
        #  [0, 1, 2, 3],
        #       ...
        #  [0, 1, 2, 3]]
        #
        index_matrx = np.array([np.arange(k),] * fft_ppg.shape[0])
        
        # edit to k+1 if  k_dominant_ppg_freqs ==  k_dominant_ppg_freqs
        index_matrx[k_dominant_ppg_freqs == k_dominant_acc_freqs] = k+1

        # take the ppg_max, 
        # which is the min value
        ppg_max_indices = index_matrx.min(axis=1)[:,None]
        ppg_max_indices[ppg_max_indices == k+1] = 0 # if all of them were the same, take the first one
        
        ppg_max = np.take_along_axis(k_dominant_ppg_freqs, ppg_max_indices, axis=1) 
        
        # take the maximum value of the ppg and acc signals
        # max freq for ppg
        #ppg_max = np.take_along_axis(ppg_freqs, np.argmax(fft_ppg, axis=1)[:,None], axis=1)

        # max freq for acc
        #acc_max = np.take_along_axis(acc_freqs, np.argmax(fft_acc, axis=1)[:,None], axis=1)

        # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
        confidence = self.calculate_confidence(ppg_freqs.copy(), fft_ppg.copy(), ppg_max)
        bpm_pred = ppg_max*60

        # load the ground_truth
        ground_truth = sp.io.loadmat(ref_fl)['BPM0'][:len(bpm_pred)]

        # measure diff
        errors = np.abs(ground_truth - bpm_pred)
        return errors[:,0], confidence
    
model = PulseRateModel()

def RunPulseRateAlgorithm(data_fl, ref_fl):
    return model.RunPulseRateAlgorithm(data_fl, ref_fl)

    100% |████████████████████████████████| 13.4MB 2.8MB/s eta 0:00:01   20% |██████▌                         | 2.7MB 30.5MB/s eta 0:00:01    30% |█████████▉                      | 4.1MB 29.3MB/s eta 0:00:01    93% |█████████████████████████████▉  | 12.5MB 29.7MB/s eta 0:00:01
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
  Found existing installation: numpy 1.12.1
    Uninstalling numpy-1.12.1:
      Successfully uninstalled numpy-1.12.1
